<a href="https://colab.research.google.com/github/uninterrupted7/NuScenesRadarData/blob/main/getSweepDataNuScenes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!mkdir -p /content/data/sets/nuscenes  # Make the directory to store the nuScenes dataset in.

In [3]:
!wget https://www.nuscenes.org/data/v1.0-mini.tgz  # Download the nuScenes mini split.

--2022-10-22 13:17:45--  https://www.nuscenes.org/data/v1.0-mini.tgz
Resolving www.nuscenes.org (www.nuscenes.org)... 99.84.192.19, 99.84.192.111, 99.84.192.114, ...
Connecting to www.nuscenes.org (www.nuscenes.org)|99.84.192.19|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4167696325 (3.9G) [application/x-tar]
Saving to: ‘v1.0-mini.tgz’

v1.0-mini.tgz       100%[===================>]   3.88G   162MB/s    in 27s     

2022-10-22 13:18:12 (147 MB/s) - ‘v1.0-mini.tgz’ saved [4167696325/4167696325]



In [4]:
!tar -xf /content/v1.0-mini.tgz -C /content/data/sets/nuscenes

In [5]:
!pip install nuscenes-devkit &> /dev/null

In [6]:
%matplotlib inline
from nuscenes.nuscenes import NuScenes

nusc = NuScenes(version='v1.0-mini', dataroot='/content/data/sets/nuscenes', verbose=True)

Loading NuScenes tables for version v1.0-mini...
23 category,
8 attribute,
4 visibility,
911 instance,
12 sensor,
120 calibrated_sensor,
31206 ego_pose,
8 log,
10 scene,
404 sample,
31206 sample_data,
18538 sample_annotation,
4 map,
Done loading in 1.021 seconds.
Reverse indexing ...
Done reverse indexing in 0.2 seconds.


In [7]:
len(nusc.sample_data)

31206

In [27]:
my_scene = nusc.scene[1]
name_scene = my_scene['name']

In [8]:
first_sample_token = my_scene['first_sample_token']

In [10]:
i=0
z=0

first_token_list = []

sensor_dict = {}

while i < len(nusc.sample_data):

  if(nusc.sample_data[i]['sample_token']==first_sample_token):
    
    first_data = nusc.sample_data[i]

    first_token_list.append([z, first_data['channel'], first_data['token']])   # to store sensor and its first token
    print(first_data)
    sensor_dict[first_data['channel']] = first_data['token']
    next_data = nusc.get('sample_data', first_data['next']) #magic
    #print(next_data)
    z+=1

  #print(i)  
  i+=1


first_token_list
sensor_dict

{'token': '37091c75b9704e0daa829ba56dfa0906', 'sample_token': 'ca9a282c9e77460f8360f564131a8af5', 'ego_pose_token': '37091c75b9704e0daa829ba56dfa0906', 'calibrated_sensor_token': 'f4d2a6c281f34a7eb8bb033d82321f79', 'timestamp': 1532402927664178, 'fileformat': 'pcd', 'is_key_frame': True, 'height': 0, 'width': 0, 'filename': 'samples/RADAR_FRONT/n015-2018-07-24-11-22-45+0800__RADAR_FRONT__1532402927664178.pcd', 'prev': '', 'next': 'f0b8593e08594a3eb1152c138b312813', 'sensor_modality': 'radar', 'channel': 'RADAR_FRONT'}
{'token': '11946c1461d14016a322916157da3c7d', 'sample_token': 'ca9a282c9e77460f8360f564131a8af5', 'ego_pose_token': '11946c1461d14016a322916157da3c7d', 'calibrated_sensor_token': '2ee327ac0903407dbb42c754861c1e63', 'timestamp': 1532402927652686, 'fileformat': 'pcd', 'is_key_frame': True, 'height': 0, 'width': 0, 'filename': 'samples/RADAR_FRONT_LEFT/n015-2018-07-24-11-22-45+0800__RADAR_FRONT_LEFT__1532402927652686.pcd', 'prev': '', 'next': 'e71dbf6026cd45178662958e95fd373

{'RADAR_FRONT': '37091c75b9704e0daa829ba56dfa0906',
 'RADAR_FRONT_LEFT': '11946c1461d14016a322916157da3c7d',
 'RADAR_FRONT_RIGHT': '491209956ee3435a9ec173dad3aaf58b',
 'RADAR_BACK_LEFT': '312aa38d0e3e4f01b3124c523e6f9776',
 'RADAR_BACK_RIGHT': '07b30d5eb6104e79be58eadf94382bc1',
 'LIDAR_TOP': '9d9bf11fb0e144c8b446d54a8a00184f',
 'CAM_FRONT': 'e3d495d4ac534d54b321f50006683844',
 'CAM_FRONT_RIGHT': 'aac7867ebf4f446395d29fbd60b63b3b',
 'CAM_BACK_RIGHT': '79dbb4460a6b40f49f9c150cb118247e',
 'CAM_BACK': '03bea5763f0f4722933508d5999c5fd8',
 'CAM_BACK_LEFT': '43893a033f9c46d4a51b5e08a67a1eb7',
 'CAM_FRONT_LEFT': 'fe5422747a7d4268a4b07fc396707b23'}

Add annotations

Add ego pos

Add sensor calibrations



In [25]:
from numpy.random.mtrand import zipf
import pickle
from nuscenes.utils.data_classes import RadarPointCloud
import numpy as np

front_radar_list = []
front_radar_left_list = []
front_radar_right_list = []
back_radar_right_list = []
back_radar_left_list = []


radar_front       = 'RADAR_FRONT'
radar_back_left   = 'RADAR_BACK_LEFT' 
radar_back_right  = 'RADAR_BACK_RIGHT'
radar_front_left  = 'RADAR_FRONT_LEFT'
radar_front_right = 'RADAR_FRONT_RIGHT'


radar_front_data        = nusc.get('sample_data', sensor_dict[radar_front])
radar_front_right_data  = nusc.get('sample_data', sensor_dict[radar_front_right])
radar_front_left_data   = nusc.get('sample_data', sensor_dict[radar_front_left])
radar_back_right_data   = nusc.get('sample_data', sensor_dict[radar_back_right])
radar_back_left_data    = nusc.get('sample_data', sensor_dict[radar_back_left])



for j in range(len(nusc.sample_data)):
  #print(next_data)
  

  
  pc = RadarPointCloud.from_file("/content/data/sets/nuscenes/" + radar_front_data['filename'])
  data = pc.points.astype(dtype=np.float32)
  front_radar_list.append(data)
  radar_front_data = nusc.get('sample_data', radar_front_data['next'])

  
  pc = RadarPointCloud.from_file("/content/data/sets/nuscenes/" + radar_front_right_data['filename'])
  data = pc.points.astype(dtype=np.float32)
  radar_front_right_data = nusc.get('sample_data', radar_front_right_data['next'])
  front_radar_right_list.append(data)


  pc = RadarPointCloud.from_file("/content/data/sets/nuscenes/" + radar_front_left_data['filename'])
  data = pc.points.astype(dtype=np.float32)
  front_radar_left_list.append(data)
  radar_front_left_data = nusc.get('sample_data', radar_front_left_data['next'])



  pc = RadarPointCloud.from_file("/content/data/sets/nuscenes/" + radar_back_right_data['filename'])
  data = pc.points.astype(dtype=np.float32)
  back_radar_right_list.append(data)
  radar_back_right_data = nusc.get('sample_data', radar_back_right_data['next'])


  
  pc = RadarPointCloud.from_file("/content/data/sets/nuscenes/" + radar_back_left_data['filename'])
  data = pc.points.astype(dtype=np.float32)
  back_radar_left_list.append(data)
  radar_back_left_data = nusc.get('sample_data', radar_back_left_data['next'])


  
  
  if (radar_front_data['next']=='' or radar_front_right_data['next']=='' or radar_front_left_data['next']=='' or radar_back_right_data['next']=='' or radar_back_left_data['next']==''):
    break

print(len(back_radar_right_list))
print(len(back_radar_left_list))
print(len(front_radar_left_list))
print(len(front_radar_right_list))
print(len(front_radar_list))

front_radar_list[0]

242
242
242
242
242


array([[  8.2,   9.6,  10.8, ...,  75.8,  81.2,  80.4],
       [  5.3,   4.3,   4.5, ...,   9.5,   8.5, -21.9],
       [  0. ,   0. ,   0. , ...,   0. ,   0. ,   0. ],
       ...,
       [  1. ,   1. ,   1. , ...,   1. ,   1. ,   1. ],
       [ 17. ,  16. ,  16. , ...,  16. ,  16. ,  16. ],
       [  3. ,   3. ,   3. , ...,   3. ,   3. ,   3. ]], dtype=float32)

In [28]:
with open('front_radar_sweeps_'+name_scene+'.pkl', 'wb') as f_rF:
  pickle.dump(front_radar_list, f_rF, protocol=2)

with open('front_radar_left_sweeps_'+name_scene+'.pkl', 'wb') as f_rFL:
  pickle.dump(front_radar_left_list, f_rFL, protocol=2)

with open('front_radar_right_sweeps_'+name_scene+'.pkl', 'wb') as f_rFR:
  pickle.dump(front_radar_right_list, f_rFR, protocol=2)

with open('back_radar_right_sweeps_'+name_scene+'.pkl', 'wb') as f_rBR:
  pickle.dump(back_radar_left_list, f_rBR, protocol=2)

with open('back_radar_left_sweeps_'+name_scene+'.pkl', 'wb') as f_rBL:
  pickle.dump(back_radar_right_list, f_rBL, protocol=2)


In [57]:
#to get annotations

s1 = nusc.get('sample', radar_front_left_data['sample_token'])
anns = s1['anns']
my_annotation_metadata =  nusc.get('sample_annotation', anns[0])
my_annotation_metadata

{'token': '3b24f083c0bf42d695a1040efdab7ffe',
 'sample_token': '67e5f88901214f3aa03d68e028185e22',
 'instance_token': '6dd2cbf4c24b4caeb625035869bca7b5',
 'visibility_token': '1',
 'attribute_tokens': ['4d8821270b4a47e3a8a300cbec48188e'],
 'translation': [373.337, 1130.661, 1.095],
 'size': [0.621, 0.669, 1.642],
 'rotation': [0.9831098797903927, 0.0, 0.0, -0.18301629506281616],
 'prev': '794fcc425f074a1392206ed925fdbbd8',
 'next': '93d5b79041c64693a5b32f1103a39a06',
 'num_lidar_pts': 0,
 'num_radar_pts': 0,
 'category_name': 'human.pedestrian.adult'}

In [48]:
print(nusc.sample_data[0])
print(nusc.sample_data[1])
print(nusc.sample_data[2])
print(nusc.sample_data[3])
print(nusc.get('sample', nusc.sample_data[0]['sample_token']))
print(nusc.get('sample', nusc.sample_data[1]['sample_token']))
print(nusc.get('sample', nusc.sample_data[2]['sample_token']))
print(nusc.get('sample', nusc.sample_data[3]['sample_token']))

{'token': '5ace90b379af485b9dcb1584b01e7212', 'sample_token': '39586f9d59004284a7114a68825e8eec', 'ego_pose_token': '5ace90b379af485b9dcb1584b01e7212', 'calibrated_sensor_token': 'f4d2a6c281f34a7eb8bb033d82321f79', 'timestamp': 1532402927814384, 'fileformat': 'pcd', 'is_key_frame': False, 'height': 0, 'width': 0, 'filename': 'sweeps/RADAR_FRONT/n015-2018-07-24-11-22-45+0800__RADAR_FRONT__1532402927814384.pcd', 'prev': 'f0b8593e08594a3eb1152c138b312813', 'next': '978db2bcdf584b799c13594a348576d2', 'sensor_modality': 'radar', 'channel': 'RADAR_FRONT'}
{'token': '37091c75b9704e0daa829ba56dfa0906', 'sample_token': 'ca9a282c9e77460f8360f564131a8af5', 'ego_pose_token': '37091c75b9704e0daa829ba56dfa0906', 'calibrated_sensor_token': 'f4d2a6c281f34a7eb8bb033d82321f79', 'timestamp': 1532402927664178, 'fileformat': 'pcd', 'is_key_frame': True, 'height': 0, 'width': 0, 'filename': 'samples/RADAR_FRONT/n015-2018-07-24-11-22-45+0800__RADAR_FRONT__1532402927664178.pcd', 'prev': '', 'next': 'f0b8593e0